# Homework 3

In [111]:
# Author: Jiechen Wu, David Alfonso

## 3

In [112]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def print_red(str):
    print(bcolors.FAIL+str+bcolors.ENDC)
    
def print_green(str):
    print(bcolors.OKGREEN+str+bcolors.ENDC)

In [113]:
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


/Users/jason.wu/miniconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['indices']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [114]:
# cercles data
circles_data = np.loadtxt(open('circles.txt','r'))
np.random.seed(0)
indices = np.random.permutation(len(circles_data[:,-1]))

# Separate into train/test as usual
circles_train_x = circles_data[indices[:1000],:-1]
circles_test_x = circles_data[indices[1000:],:-1]
circles_train_y = circles_data[indices[:1000],-1]
circles_test_y = circles_data[indices[1000:],-1]
# Remap y classes to int 1,2
circles_train_y = (circles_train_y+1).astype(int)
circles_test_y = (circles_test_y+1).astype(int)


# fashion MNIST
import utils.mnist_reader as mnist_reader
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [119]:
def rect(x):
    return np.maximum(np.zeros(len(x)),x)

def softmax(x_v):
    sum_exp = np.sum(np.exp(x_v))
    return np.exp(x_v)/sum_exp

class NeuralNetwork:
    def __init__(self,x,y,dh,m):
        self.x = x
        self.y = y
        self.d = len(x)
        self.dh = dh # number of hiden layer neurons
        self.m = m # m classes
        
        # hyperparameters
        self.lmbd11,self.lmbd12,self.lmbd21,self.lmbd22= 0.1,0.1,0.1,0.1

        # Parameter initialization
        sqrtr_nc_W1 = 1/self.d**0.5
        self.W1 = np.random.uniform(-sqrtr_nc_W1,sqrtr_nc_W1,(self.dh,self.d))
        self.b1 = np.zeros(self.dh)
        sqrtr_nc_W2 = 1/self.dh**0.5
        self.W2 = np.random.uniform(-sqrtr_nc_W2,sqrtr_nc_W2,(self.m,self.dh))
        self.b2 = np.zeros(self.m)
        self.wb2theta()
        self.gradFinitDiff = np.zeros(len(self.theta))

    def bprop(self):
        model.theta2wb()
        self.grad_oa = self.os - np.eye(self.m)[self.y-1]
        self.grad_hs = self.W2.T @ self.grad_oa
        self.grad_ha = (self.ha>0).astype(int) * self.grad_hs
        self.grad_x = self.W1.T @ self.grad_ha
        self.grad_b2 = self.grad_oa
        self.grad_W2 = np.outer(self.grad_oa, self.hs) \
        +self.lmbd21*np.sign(self.W2)+2*self.lmbd22*self.W2
        self.grad_b1 = self.grad_ha
        self.grad_W1 = np.outer(self.grad_ha, self.x) \
        +self.lmbd11*np.sign(self.W1)+2*self.lmbd12*self.W1
        self.serial_grad()

    def fprop(self,ischeck = False, check_theta = None):
        if ischeck == False:
            theta = self.theta
            W1,b1,W2,b2 = self.W1,self.b1,self.W2,self.b2
        else:
            theta = check_theta
            W1,b1,W2,b2 = self.unserial_param(check_theta)
        
        self.ha = W1@self.x+b1
        self.hs = rect(self.ha)
        self.oa = W2@self.hs+b2
        self.os = softmax(self.oa)
        self.L = -np.log(self.os[self.y-1])
        self.L_reg = self.L + self.lmbd11*np.sum(np.abs(W1)) \
        + self.lmbd12*np.sum(W1**2) \
        + self.lmbd21*np.sum(np.abs(W2)) \
        + self.lmbd22*np.sum(W2**2) 
        return self.L_reg
        
    def serial_param(self,W1,b1,W2,b2):
        return np.concatenate([np.ravel(W1),
                                     np.ravel(b1),
                                     np.ravel(W2),
                                     np.ravel(b2)])
        
    def unserial_param(self,theta):
        dh = self.dh
        d = self.d
        W1 = theta[0:dh*d].reshape((dh,d))
        b1 = theta[dh*d:dh*d+dh]
        W2 = theta[dh*d+dh:dh*d+dh+m*dh].reshape((m,dh))
        b2 = theta[-m:]
        return W1,b1,W2,b2
    
    def wb2theta(self):
        self.theta = self.serial_param(self.W1,self.b1,self.W2,self.b2)
        
    
    def theta2wb(self):
        self.W1,self.b1,self.W2,self.b2 = self.unserial_param(self.theta)
        
    def serial_grad(self):
        self.gradBprop = np.concatenate([np.ravel(self.grad_W1),
                             np.ravel(self.grad_b1),
                             np.ravel(self.grad_W2),
                             np.ravel(self.grad_b2)])

    def cal_finit_difference(self):
        epsilon = 10**-5
        for ind,t in enumerate(self.theta):
            thetaPlus = np.copy(self.theta)
            thetaPlus[ind] += epsilon
            self.gradFinitDiff[ind] = ((self.fprop(True,thetaPlus)-self.fprop())/epsilon)

    def gradient_check(self):
        sys_eps = np.nextafter(0, 1)
        ratio = (self.gradFinitDiff+sys_eps)/(self.gradBprop+sys_eps)
        if np.any(ratio>1.01) or np.any(ratio<0.99):
            print_red("Gradient check failed.")
            print("ratio=",ratio)
        else:
            print_green("Gradient check succeeded.")
    
    def show_two_grads(self):
        print("Gradient Backprop:",self.gradBprop)
        print("Gradient Finit Differenct:",self.gradFinitDiff)

### 3.1

In [120]:
dh = 2
m = 2
model = NeuralNetwork(circles_train_x[0,:],circles_train_y[0],dh,m)
model.fprop()
model.wb2theta()
model.bprop()
model.cal_finit_difference()
model.gradient_check()

Gradient check succeeded.


### 3.2

In [121]:
model.show_two_grads()

Gradient Backprop: [ 0.34845323 -0.11339211 -0.10652069  0.12420946 -0.27128457 -0.
 -0.16693179 -0.18456706  0.13623811  0.17374942  0.49925081 -0.49925081]
Gradient Finit Differenct: [ 0.34845442 -0.11339106 -0.10651969  0.12421046 -0.2712842   0.
 -0.16693079 -0.18456606  0.13623911  0.17375042  0.49925206 -0.49924956]


### 3.3 & 3.4

In [122]:
# mini batch size
K = 10
#data_batch = 
for i in range(K):
    print("#record:",i)
    model = NeuralNetwork(circles_train_x[i,:],circles_train_y[i],dh,m)
    model.fprop()
    model.wb2theta()
    model.bprop()
    model.cal_finit_difference()
    model.gradient_check()
    model.show_two_grads()

#record: 0
Gradient check succeeded.
Gradient Backprop: [-0.10843412  0.17497925  0.12471347  0.18756623 -0.04400905  0.
 -0.16596074  0.18383367 -0.18381014 -0.15705944  0.49921681 -0.49921681]
Gradient Finit Differenct: [-0.10843312  0.17498025  0.12471447  0.18756723 -0.04400904  0.
 -0.16595974  0.18383467 -0.18380914 -0.15705844  0.49921806 -0.49921556]
#record: 1
Gradient check succeeded.
Gradient Backprop: [ 0.23524751 -0.22840675 -0.18775093  0.10590428  0.          0.
 -0.13507412 -0.17294522  0.1521994   0.14786744 -0.5         0.5       ]
Gradient Finit Differenct: [ 0.23524851 -0.22840575 -0.18774993  0.10590528  0.          0.
 -0.13507312 -0.17294422  0.1522004   0.14786844 -0.49999875  0.50000125]
#record: 2
Gradient check succeeded.
Gradient Backprop: [ 0.39383561  0.56328164  0.18555777 -0.25951742  0.39158395 -0.13273091
 -0.69420669 -0.22043486  0.36470694 -0.12169572 -0.61302583  0.61302583]
Gradient Finit Differenct: [ 0.39383669  0.56328304  0.18555878 -0.25951637

### 3.5